Below is the code for scraping tournament data. We will likely convert this to a script.

In [24]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from urllib.request import urlretrieve
import os

`get_link` gets all links to games where fox is playing vs jigglypuff. We will eliminate this requirement when we go more general.

In [25]:
tournaments = {
    "ss8": ["https://slippi.gg/tournament/97437", 12],
    "genesis7": ["https://slippi.gg/tournament/111517", 2],
    "bighouse9": ["https://slippi.gg/tournament/109637", 3],
    "shine2019": ["https://slippi.gg/tournament/98817", 2],
    "sns5": ["https://slippi.gg/tournament/27017", 2],
    "goml2019": ["https://slippi.gg/tournament/12906", 3],
    "pound2019": ["https://slippi.gg/tournament/25852", 2]
}

In [26]:
def get_link(g):
    # Get the image icons of the characters
    icons = g.find_elements_by_css_selector("img")
    # Get the icon ids based on the source image. This shows which characters are being played.
    icon_ids = sorted([int(icon.get_attribute("src").split("icon")[1].split("-")[1]) for icon in icons])
    # Return None if it's not Fox vs Puff
    if icon_ids != [2, 15]:
        return None
    # Else return the link to the game.
    return g.find_element_by_css_selector("a").get_attribute("href")

Starts the selenium web driver.

In [27]:
driver = webdriver.Firefox()

The main code: goes through each tournament page and gets all links.

In [28]:
for tournament in tournaments:
    link, pages = tournaments[tournament]
    links = []
    for i in range(pages):
        driver.get(link + "?page={}".format(i+1))
        delay = 10
        try:
            myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tbody')))
        except TimeoutException:
            print("Loading took too much time!")
        games = driver.find_element_by_css_selector("tbody").find_elements_by_css_selector("tr")
        cur_links = [get_link(game) for game in games]
        cur_links = [link for link in cur_links if link != None]
        links += cur_links
    outdir = os.path.join("./slip", tournament)
    os.makedirs(outdir, exist_ok=True)
    for link in links:
        urlretrieve(link, os.path.join(outdir, link.split("/")[-1]))
    print("Finished tournament {}".format(tournament))

Finished tournament ss8
Finished tournament genesis7
Finished tournament bighouse9
Finished tournament shine2019
Finished tournament sns5
Finished tournament goml2019
Finished tournament pound2019
